In [66]:
import time
#import math
import pyodbc
import pandas as pd
from datetime import date, datetime
import numpy as np
import datetime

In [67]:
today=datetime.date.today()

In [187]:
conn=pyodbc.connect('dsn=NetSuite;UID=RHung@Top-Line.com;PWD=Netsuite888')
sqld=datetime.date.today().strftime("%Y/%m/%d")
ALLdata="""
select 
              items.name as name,
              transactions.trandate as 'Sales',
              locations.name as Destination,
              -(transaction_lines.item_count) as "sales amount",
              items.TYPE_NAME,
              transactions.entity_id as "entity_id"
from
              transaction_lines
               left join transactions on transactions.transaction_id=transaction_lines.transaction_id
               left join items on items.item_id=transaction_lines.item_id
               left join locations on locations.location_id=transaction_lines.location_id
               
where
              transaction_lines.item_count < 0 and
              transaction_lines.ACCOUNT_ID=123 and
              transactions.entity_id is not null and
              transactions.transaction_type = \'Item Fulfillment\' and
              transactions.status = \'Shipped\' and
              transactions.trandate < \'"""+sqld+"""\' and
              transactions.trandate > '2020/01/01' and
              items.isinactive = 'No' and
              items.STATUS_ID is not null and
              items.REPLACED_FOR_OLD_ITEM is NULL    
order by items.name
"""

kit='''
select
imem.name as'member',
i.name as 'kit',
ig.quantity as 'num'

from 
items i
left join item_group ig on ig.parent_id=i.item_id
left join items imem on imem.item_id=ig.member_id
where
i.isinactive = 'No' and
i.type_name = 'Kit/Package'

'''



In [188]:
sosql="""
select    
    items.name as "Item",
    transactions.trandate as 'trandate',
    transactions.TRANID as 'SO doc number',

    -transaction_lines.item_count as 'quantity'


from
    items
    left join transaction_lines on items.item_id=transaction_lines.item_id
    left join transactions on transactions.transaction_id=transaction_lines.transaction_id
    left join locations on locations.location_id=transaction_lines.location_id
where

    transactions.transaction_type = \'Sales Order\' and
    transactions.trandate < \'"""+sqld+"""\' and
    transactions.trandate > '2020/08/01'  and
    items.type_name = 'Inventory Item' and
    transaction_lines.item_count <0
  
"""

In [70]:
items=pd.read_sql(ALLdata,conn)
kits=pd.read_sql(kit,conn)


In [189]:
so=pd.read_sql(sosql,conn)


In [246]:
kits
kitssum=pd.DataFrame(kits.groupby(['kit'],as_index=True).sum()['num'])
kits=pd.merge(kits,kitssum, how='left',on='kit')

In [247]:
kits

,member,kit,num_x,num_y
0,E208C-GV2,E208S-GV[4S],1.0,4.0
1,E208LS-GV2,E208S-GV[4S],1.0,4.0
2,E208C-GV1,E208S-GV[4S],1.0,4.0
3,E208CE-GV,E208S-GV[4S],1.0,4.0
4,E208C-GV1,E208S-GV[5S],1.0,4.0
...,...,...,...,...
16133,E217GAC-DGL(N1),E217GAC-DGL[2PC],1.0,1.0
16134,E995CR-GL(SPK),E995GL[CHL],1.0,2.0
16135,E995OT-GL(SPK),E995GL[CHL],1.0,2.0
16136,E592-B1BK(N1),403-62TR[TBL]B1BK,1.0,2.0


In [61]:
so.sort_values(by=['SO doc number'])

,Item,trandate,SO doc number,quantity
10034,E318DSF-GLA(NEW),2020-08-05,S200716788,1.0
12823,E390F-1(3A),2020-08-06,S200725323,1.0
2844,E638BF-3CP(3A),2020-08-03,S200729944,1.0
24297,E390F-1(3A),2020-08-10,S200730665,1.0
24298,E390F-1(3A),2020-08-10,S200730790,1.0
...,...,...,...,...
43000,E784BKL-2GL,2020-08-17,S200830063,1.0
43002,579BR-15(3A),2020-08-17,S200830064,1.0
44169,E351-12L(3A),2020-08-17,S200830082,1.0
42430,2601-48(3A),2020-08-17,S200830086,1.0


In [90]:
so_list=list(so['SO doc number'].unique())

In [195]:
#篩選出訂單只有一個item quantity也=1 不可能為kit的item
itemorder=pd.DataFrame()
kitorder=pd.DataFrame()
for i in range(len(so_list)):
    condition=so[(so['SO doc number'] == so_list[i])& (so['quantity'] == 1)]
    if condition.shape[0] == 1:
        itemorder=itemorder.append(condition)
    else:
        kitorder=kitorder.append(condition)

In [251]:
kitorder_list=list(kitorder['SO doc number'].unique())

In [252]:
kitorder

,Item,trandate,SO doc number,quantity
8,2515-96B(3A),2020-08-02,S200776704,1.0
10,2515-96T(MTL),2020-08-02,S200776704,1.0
14,E637B-A(3A),2020-08-02,S200776873,1.0
15,E637B-B(3A),2020-08-02,S200776873,1.0
17,E802-04B,2020-08-02,S200776880,1.0
...,...,...,...,...
49834,245BTF-3WH,2020-08-19,S200833450,1.0
49835,E557-42WH,2020-08-19,S200832440,1.0
49836,E557-42BWH,2020-08-19,S200832440,1.0
49848,2058CPRF-2,2020-08-19,S200832464,1.0


In [249]:
left=pd.merge(kitorder,kits, how='left',left_on='Item', right_on='member')
#outer.to_csv('outer.csv')

In [254]:
kitorder_list=list(left['SO doc number'].unique())

In [257]:
left_nokit=left[left['kit'].isnull()] #沒有跟kit merge到的

In [258]:
left=left[left['kit'].notnull()]#有跟kit merge到的
left

,Item,trandate,SO doc number,quantity,member,kit,num_x,num_y
0,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96(MTL)[TBL],1.0,2.0
1,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[5PC],1.0,4.0
2,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[6PC],1.0,5.0
3,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[7PC],1.0,5.0
4,2515-96T(MTL),2020-08-02,S200776704,1.0,2515-96T(MTL),2515-96[5PC],1.0,4.0
...,...,...,...,...,...,...,...,...
124720,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5NK]C4,1.0,5.0
124721,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]C314WH,1.0,5.0
124722,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]C214WH,1.0,5.0
124723,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]551BST,1.0,4.0


In [259]:
left['count']=left['quantity']/left['num_x']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [260]:
left

,Item,trandate,SO doc number,quantity,member,kit,num_x,num_y,count
0,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96(MTL)[TBL],1.0,2.0,1.0
1,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[5PC],1.0,4.0,1.0
2,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[6PC],1.0,5.0,1.0
3,2515-96B(3A),2020-08-02,S200776704,1.0,2515-96B(3A),2515-96[7PC],1.0,5.0,1.0
4,2515-96T(MTL),2020-08-02,S200776704,1.0,2515-96T(MTL),2515-96[5PC],1.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...
124720,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5NK]C4,1.0,5.0,1.0
124721,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]C314WH,1.0,5.0,1.0
124722,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]C214WH,1.0,5.0,1.0
124723,E557-42BWH,2020-08-19,S200832440,1.0,E557-42BWH,E557-42WH[5PC]551BST,1.0,4.0,1.0


In [261]:
left.drop_duplicates(subset=None, keep='first', inplace=True)
left_s=pd.DataFrame(left.groupby(['SO doc number','kit'],as_index=True).sum()['count'])
#left_s.drop_duplicates(subset=None, keep='first', inplace=True)
left_s=left_s.sort_values(by=['SO doc number'])
left_s=left_s.reset_index()
#left_s.drop_duplicates(subset='SO doc number', keep='first', inplace=True)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [265]:
left_s
result=pd.merge(left_s,kits, how='left',on='kit')

In [268]:
result

,SO doc number,kit,count,member,num_x,num_y
0,S200733757,E318DSF-GLA(3A)[BD],1.0,E318DSF-GLA(NEW),1.0,3.0
1,S200733757,E318DSF-GLA(3A)[BD],1.0,E318DSF-GLB(NEW),1.0,3.0
2,S200733757,E318DSF-GLA(3A)[BD],1.0,E318B-CGL(NEW),1.0,3.0
3,S200733757,E318DSF-GLA(3A)[BD]AB,1.0,E318DSF-GLA(NEW),1.0,2.0
4,S200733757,E318DSF-GLA(3A)[BD]AB,1.0,E318DSF-GLBN(NEW),1.0,2.0
...,...,...,...,...,...,...
263668,S200833533,530-60BAQ(3A)[5PC]C3AQ,1.0,530-BAQ(3A),1.0,4.0
263669,S200833533,530-60BAQ(3A)[5PC]C3AQ,1.0,530C3-AQ(3A),2.0,4.0
263670,S200833533,530-60BWH[5PC]C5WH,1.0,530-60AK(MTL),1.0,4.0
263671,S200833533,530-60BWH[5PC]C5WH,1.0,530-BWH(3A),1.0,4.0


In [278]:
result['equal'] = np.where(result['count'] == result['num_y'], 'YES', 'NO')
result

,SO doc number,kit,count,member,num_x,num_y,equal
0,S200733757,E318DSF-GLA(3A)[BD],1.0,E318DSF-GLA(NEW),1.0,3.0,NO
1,S200733757,E318DSF-GLA(3A)[BD],1.0,E318DSF-GLB(NEW),1.0,3.0,NO
2,S200733757,E318DSF-GLA(3A)[BD],1.0,E318B-CGL(NEW),1.0,3.0,NO
3,S200733757,E318DSF-GLA(3A)[BD]AB,1.0,E318DSF-GLA(NEW),1.0,2.0,NO
4,S200733757,E318DSF-GLA(3A)[BD]AB,1.0,E318DSF-GLBN(NEW),1.0,2.0,NO
...,...,...,...,...,...,...,...
263668,S200833533,530-60BAQ(3A)[5PC]C3AQ,1.0,530-BAQ(3A),1.0,4.0,NO
263669,S200833533,530-60BAQ(3A)[5PC]C3AQ,1.0,530C3-AQ(3A),2.0,4.0,NO
263670,S200833533,530-60BWH[5PC]C5WH,1.0,530-60AK(MTL),1.0,4.0,NO
263671,S200833533,530-60BWH[5PC]C5WH,1.0,530-BWH(3A),1.0,4.0,NO


In [284]:
result_y=result[result['equal']=='YES'].drop_duplicates(subset='member', keep='last', inplace=False)

In [271]:
result.apply(lambda x : func(result['count'],result['num_y']),axis = 1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [166]:
left_s[left_s['SO doc number']=='S200777543']

,SO doc number,kit,count


In [242]:
left_s.to_csv('left_s0824.csv')

In [122]:
left_s=left_s.reset_index()
left_s.drop_duplicates(subset=None, keep='first', inplace=True)

In [46]:
left[left['SO doc number']=='S200739420']

,Item,trandate,SO doc number,quantity,member,kit,num,count
39639,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2DGL[BED]PL,1.0,1.0
39640,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2GL[BED]PL,1.0,1.0
39641,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2WL[BED]PL,1.0,1.0
39642,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2DGL[BED],1.0,1.0
39643,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2GL[BED],1.0,1.0
39644,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2WL[BED],1.0,1.0
39645,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2DGL[HB],1.0,1.0
39646,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2GL[HB],1.0,1.0
39647,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,315BKL-2WL[HB],1.0,1.0
39648,E386BKL-1,2020-08-04,S200739420,1.0,E386BKL-1,E784BKL-2BL[HB],1.0,1.0


In [49]:
left

,Item,trandate,SO doc number,quantity,member,kit,num,count
0,3270-24S1BK(3A),2020-08-02,S200762967,1.0,3270-24S1BK(3A),3270-24S1BK[2PC],1.0,1.000000
1,E720A-ES(N1),2020-08-02,S200776548,1.0,NaN,NaN,NaN,NaN
2,E355C-BL(3A),2020-08-02,S200776589,1.0,NaN,NaN,NaN,NaN
3,530-BBK(3A),2020-08-02,S200776590,1.0,530-BBK(3A),530-45BBK(3A)[TBL],1.0,1.000000
4,530-BBK(3A),2020-08-02,S200776590,1.0,530-BBK(3A),530-60BBK(MTL)[TBL],1.0,1.000000
...,...,...,...,...,...,...,...,...
247658,E848C1-BL,2020-08-16,S200827961,1.0,E848C1-BL,5428NT-84[5PC]C1BL,2.0,0.500000
247659,E848C1-BL,2020-08-16,S200827961,1.0,E848C1-BL,5428NT-45RD[5PC]C1BL,2.0,0.500000
247660,E848C1-BL,2020-08-16,S200827961,1.0,E848C1-BL,5428NT-45RD[3PC]C1BL,1.0,1.000000
247661,E848C1-BL,2020-08-16,S200827961,1.0,E848C1-BL,E431-60RDB[5PC]848BL,2.0,0.500000


In [55]:
left_s.to_csv('left_s0818.csv')

In [31]:
len(left['SO doc number'].unique())

505114

In [5]:
pd.DataFrame(items.groupby(['name','Sales']).sum()['sales amount']).head(50)

sales amount
name    Sales                   
032A-GA 2020-01-02           1.0
        2020-01-05           1.0
        2020-01-06           1.0
        2020-01-16           1.0
        2020-01-19           1.0
        2020-02-03           1.0
        2020-02-21           1.0
        2020-02-23           1.0
        2020-03-02           1.0
        2020-03-04           1.0
        2020-03-09           2.0
        2020-03-11           1.0
        2020-03-16           1.0
        2020-03-17           1.0
        2020-03-24           1.0
        2020-03-30           2.0
        2020-04-02           1.0
        2020-04-07           1.0
        2020-04-10           2.0
        2020-04-13           1.0
        2020-04-16           1.0
        2020-04-17           3.0
        2020-04-20           2.0
        2020-04-21           1.0
        2020-04-22           1.0
        2020-04-26           2.0
        2020-04-28           2.0
        2020-04-29           2.0
        2020-04-30           2.0
        2020-05-03           1.0
        2020-05-05           1.0
        2020-05-06           2.0
        2020-05-07           1.0
        2020-05-08           1.0
033B-4  2020-01-02           6.0
        2020-01-06           1.0
        2020-01-09           3.0
        2020-01-10           2.0
        2020-01-12           1.0
        2020-01-14           3.0
        2020-01-17           1.0
        2020-01-21           2.0
        2020-01-23           2.0
        2020-01-27           2.0
        2020-01-31           4.0
        2020-02-03           5.0
        2020-02-07           1.0
        2020-02-12           2.0
        2020-03-23           4.0
        2020-03-24           2.0

In [33]:
kits

,kit,member,num,dic
0,E208S-GV[4S],E208C-GV2,1.0,"""E208C-GV2"":1.0"
1,E208S-GV[4S],E208LS-GV2,1.0,"""E208LS-GV2"":1.0"
2,E208S-GV[4S],E208C-GV1,1.0,"""E208C-GV1"":1.0"
3,E208S-GV[4S],E208CE-GV,1.0,"""E208CE-GV"":1.0"
4,E208S-GV[5S],E208C-GV1,1.0,"""E208C-GV1"":1.0"
...,...,...,...,...
16098,E217GAC-DGL[2PC],E217GAC-DGL(N1),2.0,"""E217GAC-DGL(N1)"":2.0"
16099,E995GL[CHL],E995CR-GL(SPK),1.0,"""E995CR-GL(SPK)"":1.0"
16100,E995GL[CHL],E995OT-GL(SPK),1.0,"""E995OT-GL(SPK)"":1.0"
16101,403-62TR[TBL]B1BK,E592-B1BK(N1),1.0,"""E592-B1BK(N1)"":1.0"


In [32]:
pd.DataFrame(kits.groupby(['kit']).apply(list))

,0
kit,
033B-7[DR],"[kit, member, num, dic]"
034WH-30[CT],"[kit, member, num, dic]"
034WN-30[CT],"[kit, member, num, dic]"
035A[SB],"[kit, member, num, dic]"
035B-7[DR],"[kit, member, num, dic]"
...,...
E995DGL[SEC]SL,"[kit, member, num, dic]"
E995GL[CHL],"[kit, member, num, dic]"
E995GL[SEC],"[kit, member, num, dic]"


In [9]:
kitscl=kits[['kit','member']]
kitss=list(kits['kit'])

In [8]:
kits['dic']='"'+kits['member']+'":'+kits['num'].map(str)


In [9]:
kitscl=kits[['kit','dic']]

In [10]:
a=kitscl.groupby('kit')['dic']
a.groupby('kit').apply(dict)

kit                         
033B-7[DR]    033B-7[DR]                 ["033B-7A":1.0, "033B-7B":1.0]
              034WH-30[CT]                                          NaN
              034WN-30[CT]                                          NaN
              035A[SB]                                              NaN
              035B-7[DR]                                            NaN
                                                 ...                   
S4513-5[D+M]  E995DGL[SEC]SL                                        NaN
              E995GL[CHL]                                           NaN
              E995GL[SEC]                                           NaN
              E995GL[SEC]SL                                         NaN
              S4513-5[D+M]      ["S4513-5(MTL)":1.0, "S4513-6(3A)":1.0]
Name: dic, Length: 35952016, dtype: object

In [39]:
pd.DataFrame(kitscl.groupby('kit')['dic']).to_dict()

{0: {0: '033B-7[DR]',
  1: '034WH-30[CT]',
  2: '034WN-30[CT]',
  3: '035A[SB]',
  4: '035B-7[DR]',
  5: '035B-9[DR]',
  6: '036B-5[DR]',
  7: '037WN-T1[TV]',
  8: '038BR-T1[TV]',
  9: '039GA-40[SV]',
  10: '040ES-40[SV]',
  11: '041WN-15[DSK]',
  12: '044WH-9[DR]',
  13: '046BR-40[SV]',
  14: '047B-5[DR]',
  15: '048-BK[CRL]',
  16: '048-TN[CRL]',
  17: '049-BK[3CH]',
  18: '055B-5[DR]',
  19: '056B-40GA[SV]',
  20: '056B-5GA[DR]',
  21: '056B-5WG[DR]',
  22: '056B-9[DR]',
  23: '056BQ-1[BED]',
  24: '057B-5[DR]',
  25: '058-40WH[SV]',
  26: '058W-WH[SB]',
  27: '060-40[SV]',
  28: '060-T1L[TV]',
  29: '060-T1[TV]',
  30: '060B-9[DR]',
  31: '061B-5[DR]',
  32: '062B-5[DR]',
  33: '063-15[DSK]',
  34: '064B-5[DR]',
  35: '065B-5[DR]',
  36: '066B-7[DR]',
  37: '067B-5[DR]',
  38: '068B-9[DR]',
  39: '069-40[SV]',
  40: '070-40[SV]',
  41: '071-40[SV]',
  42: '074DC-04[E+C+S]',
  43: '074DC-04[E+C]',
  44: '075WN-04[E+C+S]',
  45: '075WN-04[E+C]',
  46: '078B-5[DR]',
  47: '079BQ-1[BED

In [40]:
b=a.to_dict()

In [45]:
b["174WNS-BF"]

KeyError: '174WNS-BF'

In [17]:
kitscl.to_dict()

{'kit': {0: 'E208S-GV[4S]',
  1: 'E208S-GV[4S]',
  2: 'E208S-GV[4S]',
  3: 'E208S-GV[4S]',
  4: 'E208S-GV[5S]',
  5: 'E208S-GV[5S]',
  6: 'E208S-GV[5S]',
  7: 'E208S-GV[6S]',
  8: 'E208S-GV[6S]',
  9: 'E208S-GV[6S]',
  10: 'E208S-GV[6S]',
  11: 'E208S-GV[6S]',
  12: '768-04[E+C]',
  13: '768-04[E+C]',
  14: '768-04[E+C+S]',
  15: '768-04[E+C+S]',
  16: '768-04[E+C+S]',
  17: '1810-1[BED]GL',
  18: '1810-1[BED]GL',
  19: '1810-1[BED]GL',
  20: '1810T-1[BED]DGL',
  21: '1810T-1[BED]DGL',
  22: '1810T-1[BED]DGL',
  23: '530-36OV-GA[TBL]',
  24: '530-36OV-GA[TBL]',
  25: '530-36OV-WH[TBL]',
  26: '530-36OV-WH[TBL]',
  27: '530-36OV-RD[TBL]',
  28: '530-36OV-RD[TBL]',
  29: '530-36OV-BK[TBL]',
  30: '530-36OV-BK[TBL]',
  31: '530-36OV-AK[TBL]',
  32: '530-36OV-AK[TBL]',
  33: '530-36GA[TBL]',
  34: '530-36GA[TBL]',
  35: '530-36WH[TBL]',
  36: '530-36WH[TBL]',
  37: '530-36AQ[TBL]',
  38: '530-36AQ[TBL]',
  39: '530-36RD[TBL]',
  40: '530-36RD[TBL]',
  41: 'E678-78[6PC]GV',
  42: 'E678-78[6

In [11]:
kitss1=list(filter(lambda x: 'PC' not in x, kitss))

NameError: name 'kitss' is not defined

In [19]:
dict(kitscl['dic'])

{0: '"E208C-GV2":1.0',
 1: '"E208LS-GV2":1.0',
 2: '"E208C-GV1":1.0',
 3: '"E208CE-GV":1.0',
 4: '"E208C-GV1":1.0',
 5: '"E208LS-GV2":2.0',
 6: '"E208CE-GV":1.0',
 7: '"E208C-GV1":1.0',
 8: '"E208S-GV2":1.0',
 9: '"E208S-GV3":1.0',
 10: '"E208LS-GV2":1.0',
 11: '"E208CE-GV":1.0',
 12: '"768-30":1.0',
 13: '"768-04":1.0',
 14: '"768-30":1.0',
 15: '"768-04":1.0',
 16: '"768-05":1.0',
 17: '"1810-1(SPK)":1.0',
 18: '"1810-2GL":1.0',
 19: '"1810-3(SPK)":1.0',
 20: '"1810T-1(SPK)":1.0',
 21: '"1810T-2DGL":1.0',
 22: '"1810T-3(SPK)":1.0',
 23: '"530-36OVAK":1.0',
 24: '"530-36XLBGA":1.0',
 25: '"530-36OVAK":1.0',
 26: '"530-36XLBWH":1.0',
 27: '"530-36OVAK":1.0',
 28: '"530-36XLBRD":1.0',
 29: '"530-36OVAK":1.0',
 30: '"530-36XLBBK":1.0',
 31: '"530-36OVAK":1.0',
 32: '"530-36XLBAK":1.0',
 33: '"530-36AK":1.0',
 34: '"530-36BGA":1.0',
 35: '"530-36AK":1.0',
 36: '"530-36BWH":1.0',
 37: '"530-36AK":1.0',
 38: '"530-36BAQ":1.0',
 39: '"530-36AK":1.0',
 40: '"530-36BRD":1.0',
 41: '"E678-78":1

In [11]:
kitss1

['E208S-GV[4S]',
 'E208S-GV[4S]',
 'E208S-GV[4S]',
 'E208S-GV[4S]',
 'E208S-GV[5S]',
 'E208S-GV[5S]',
 'E208S-GV[5S]',
 'E208S-GV[6S]',
 'E208S-GV[6S]',
 'E208S-GV[6S]',
 'E208S-GV[6S]',
 'E208S-GV[6S]',
 '768-04[E+C]',
 '768-04[E+C]',
 '768-04[E+C+S]',
 '768-04[E+C+S]',
 '768-04[E+C+S]',
 '1810-1[BED]GL',
 '1810-1[BED]GL',
 '1810-1[BED]GL',
 '1810T-1[BED]DGL',
 '1810T-1[BED]DGL',
 '1810T-1[BED]DGL',
 '530-36OV-GA[TBL]',
 '530-36OV-GA[TBL]',
 '530-36OV-WH[TBL]',
 '530-36OV-WH[TBL]',
 '530-36OV-RD[TBL]',
 '530-36OV-RD[TBL]',
 '530-36OV-BK[TBL]',
 '530-36OV-BK[TBL]',
 '530-36OV-AK[TBL]',
 '530-36OV-AK[TBL]',
 '530-36GA[TBL]',
 '530-36GA[TBL]',
 '530-36WH[TBL]',
 '530-36WH[TBL]',
 '530-36AQ[TBL]',
 '530-36AQ[TBL]',
 '530-36RD[TBL]',
 '530-36RD[TBL]',
 'E938CA-1B(3A)[C+O]',
 'E938CA-1B(3A)[C+O]',
 'E938CA-1B(3A)[C+O]',
 'E938CA-1B(3A)[C+O]',
 'E938CH-1B(3A)[C+O]',
 'E938CH-1B(3A)[C+O]',
 'E938CH-1B(3A)[C+O]',
 'E938CH-1B(3A)[C+O]',
 '9913GL-1TL[C+O]',
 '9913GL-3TLA[SOFA]',
 '9913GL-1TL[C+O

In [12]:
SOpd=pd.DataFrame(so.groupby(['SO doc number','Item']).sum()['quantity'])

In [30]:
so[so['Item'].isin(kitss1)]

,Item,trandate,SO doc number,Name,loc,quantity,memo
74,4956PUWH-[BD]C,2020-02-05,S200203052,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/07/2020
180,4513F-1[BED],2020-02-24,S200233099,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/26/2020
190,E411B221W(3A)[BED],2020-04-29,S200494106,BedBathBeyond.com,IL-S,1.0,PODate 04/29/2020 PODate 04/29/2020
267,4513F-1[BED],2020-03-31,S200363517,Wal-Mart Stores Inc (Dot Com) : Wal-Mart.com (...,CA-F,1.0,Customer Order Date:03/31/2020 Delivery Reques...
286,4956PU-[BD]C,2020-07-13,S200732615,Target Corporation : Target.com (Drop Ship),IL-S,1.0,Cancel After Date:07/17/2020 Sold Date:07/13/2...
...,...,...,...,...,...,...,...
737028,E938CA-1B(3A)[CHR],2020-06-22,S200646088,Target Corporation : Target.com (Drop Ship),IL-S,1.0,Cancel After Date:06/26/2020 Sold Date:06/22/2...
737080,4513Q-1[BED],2020-02-28,S200240960,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:03/03/2020
737106,E208B-AGL(3A)[BD],2020-05-07,S200519264,BedBathBeyond.com,CA-F,1.0,PODate 05/07/2020 PODate 05/07/2020
737123,5118W-42(3A)[TBL],2020-04-20,S200458411,Wal-Mart Stores Inc (Dot Com) : Wal-Mart.com (...,CA-F,1.0,Customer Order Date:04/20/2020 Delivery Reques...


In [13]:
def fil(data):
    item=list(data[data['Item'].isin(kitss)]['SO doc number'])#裡面有kits的訂單號
    order=item[item['SO doc number'].isin(item)]#用訂單號去濾有kit的訂單

In [55]:
#如果A欄是xxx，return他在B欄的值
#def kit(a):
items=list(kits[kits['kit'].isin(['4513F-1[BED]'])]['member'])#回傳list
num=list(kits[kits['kit'].isin(['4513F-1[BED]'])]['num'])#會傳個數

In [77]:

so[so['Item'].isin(['530-60BAK(MTL)[TBL]'])]['SO doc number']

7204      S200601611
132127    S200516933
143178    S200143266
172513    S200591218
173522    S200642554
264047    S200226705
553811    S200334894
561498    S200137316
623034    S200316124
671906    S200132746
673434    S200767154
731852    S200661616
Name: SO doc number, dtype: object

In [37]:
item=list(so[so['Item'].isin(kitss1)]['SO doc number'])#裡面有kits的訂單號
order=so[so['SO doc number'].isin(item)]
order

,Item,trandate,SO doc number,Name,loc,quantity,memo
74,4956PUWH-[BD]C,2020-02-05,S200203052,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/07/2020
75,4956PUWH-A(3A),2020-02-05,S200203052,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/07/2020
76,4956PUWH-B(3A),2020-02-05,S200203052,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/07/2020
77,4956PUWH-C(3A),2020-02-05,S200203052,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/07/2020
180,4513F-1[BED],2020-02-24,S200233099,"Hayneedle (Net Shops, Inc.)",CA-F,1.0,Requested Ship Date:02/26/2020
...,...,...,...,...,...,...,...
737124,5118-42(3A),2020-04-20,S200458411,Wal-Mart Stores Inc (Dot Com) : Wal-Mart.com (...,CA-F,1.0,Customer Order Date:04/20/2020 Delivery Reques...
737125,5118W-42B(3A),2020-04-20,S200458411,Wal-Mart Stores Inc (Dot Com) : Wal-Mart.com (...,CA-F,1.0,Customer Order Date:04/20/2020 Delivery Reques...
737238,E425-30SA(3A)[CKTL],2020-01-11,S200115561,BedBathBeyond.com,CA-F,1.0,PODate 01/11/2020 PODate 01/11/2020
737239,E425-30SA(3A),2020-01-11,S200115561,BedBathBeyond.com,CA-F,1.0,PODate 01/11/2020 PODate 01/11/2020


In [27]:
item

['S200203052',
 'S200233099',
 'S200494106',
 'S200363517',
 'S200732615',
 'S200407629',
 'S200340700',
 'S200492704',
 'S200331899',
 'S200530917',
 'S200460595',
 'S200141755',
 'S200426504',
 'S200654121',
 'S200401211',
 'S200112070',
 'S200344900',
 'S200201241',
 'S200442476',
 'S200510170',
 'S200712578',
 'S200763652',
 'S200121068',
 'S200449812',
 'S200643881',
 'S200659155',
 'S200533829',
 'S200219641',
 'S200508153',
 'S200425643',
 'S200769638',
 'S200225450',
 'S200135736',
 'S200445515',
 'S200626486',
 'S200767870',
 'S200515036',
 'S200632891',
 'S200134898',
 'S200127854',
 'S200659250',
 'S200741267',
 'S200484236',
 'S200214339',
 'S200590259',
 'S200234384',
 'S200525503',
 'S200666618',
 'S200702434',
 'S200228118',
 'S200231018',
 'S200204976',
 'S200613706',
 'S200137415',
 'S200576258',
 'S200665033',
 'S200743327',
 'S200227239',
 'S200334652',
 'S200500965',
 'S200118660',
 'S200629478',
 'S200606827',
 'S200736038',
 'S200631089',
 'S200601611',
 'S2001419

In [51]:
order.to_csv('order.csv')

In [104]:
me=pd.merge(so,kitscl,how='inner',left_on='Item',right_on='kit')
me

,Item,trandate,SO doc number,Name,loc,quantity,memo,kit,member
0,530C1-24AK[2PC],2020-02-09,S200208522,Overstock.Com,CA-F,2.0,PODate 02/09/2020,530C1-24AK[2PC],530C1-24AK
1,530C1-24AK[2PC],2020-01-31,S200147707,Overstock.Com,CA-F,1.0,PODate 01/31/2020,530C1-24AK[2PC],530C1-24AK
2,530C1-24AK[2PC],2020-04-02,S200402324,Overstock.Com,CA-F,2.0,PODate 04/02/2020,530C1-24AK[2PC],530C1-24AK
3,530C1-24AK[2PC],2020-02-24,S200235186,Overstock.Com,CA-F,3.0,PODate 02/24/2020,530C1-24AK[2PC],530C1-24AK
4,530C1-24AK[2PC],2020-02-29,S200241134,Overstock.Com,CA-F,1.0,PODate 02/29/2020,530C1-24AK[2PC],530C1-24AK
...,...,...,...,...,...,...,...,...,...
71894,530-45BAQ(3A)[TBL],2020-04-27,S200487269,Target Corporation : Target.com (Drop Ship),CA-F,1.0,Cancel After Date:05/01/2020 Sold Date:04/27/2...,530-45BAQ(3A)[TBL],530-45AK(3A)
71895,530-45BAQ(3A)[TBL],2020-04-27,S200487269,Target Corporation : Target.com (Drop Ship),CA-F,1.0,Cancel After Date:05/01/2020 Sold Date:04/27/2...,530-45BAQ(3A)[TBL],530-BAQ(3A)
71896,2515-96[5PC],2020-01-07,S200109713,"Hayneedle (Net Shops, Inc.)",IL-S,1.0,Requested Ship Date:01/14/2020,2515-96[5PC],2515-96T(MTL)
71897,2515-96[5PC],2020-01-07,S200109713,"Hayneedle (Net Shops, Inc.)",IL-S,1.0,Requested Ship Date:01/14/2020,2515-96[5PC],2515-96B(3A)


In [103]:
me[['SO doc number']]

,SO doc number
0,S200208522
1,S200147707
2,S200402324
3,S200235186
4,S200241134
...,...
71894,S200487269
71895,S200487269
71896,S200109713
71897,S200109713


In [51]:
me

NameError: name 'me' is not defined

In [52]:
so

,Item,trandate,SO doc number,Name,loc,quantity,memo
0,530SW5-24AK,2020-03-09,S200313909,Wayfair.com : Castlegate - KY Hebron,CG-ER,1.0,PODate 03/09/2020 RequestedShipDate 03/09/2020...
1,530SW5-24AK,2020-03-09,S200313909,Wayfair.com : Castlegate - KY Hebron,CG-ER,1.0,PODate 03/09/2020 RequestedShipDate 03/09/2020...
2,122D200W(3A),2020-05-13,S200537778,Wayfair.com : Castlegate - GA McDonough,CG-ER,1.0,PODate 05/13/2020 RequestedShipDate 06/09/2020...
3,E557-60RD,2020-03-09,S200314638,Wayfair.com,CA-F,1.0,PODate 03/09/2020 RequestedShipDate 03/10/2020...
4,664WN-24SB[2PC]DGL,2020-02-22,S200230502,Target Corporation : Target.com (Drop Ship),CA-F,1.0,Cancel After Date:02/25/2020 Sold Date:02/22/2...
...,...,...,...,...,...,...,...
737257,681A-3V(3A),2020-01-16,S200123086,Wayfair.com,CA-F,3.0,PODate 01/16/2020 RequestedShipDate 01/16/2020...
737258,5106BK,2020-06-08,S200614998,Wayfair.com : Castlegate - GA Savannah,CG-ER,1.0,PODate 06/08/2020 RequestedShipDate 06/10/2020...
737259,3228-15C(3A),2020-02-17,S200220431,Wayfair.com : Castlegate - NJ Cranbury,CG-ER,1.0,PODate 02/17/2020 RequestedShipDate 02/17/2020...
737260,E713A-V(3A),2020-05-02,S200504890,Overstock.Com,IL-S,1.0,PODate 05/02/2020


In [ ]:
#look up the key from value in dictionary
dictionary = {'george': 16, 'amber': 19}
search_age = input("Provide age")
for name, age in dictionary.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if age == search_age:
        print(name)